In [17]:
# Imports
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import TimeoutException
import undetected_chromedriver as uc

import os
import time
from bs4 import BeautifulSoup
import re
import pandas as pd
import requests

In [18]:
# Configure Chrome Webdriver

chrome_install = ChromeDriverManager().install()

In [19]:
# Initialize Chrome WebDriver
options = webdriver.ChromeOptions()
#options.add_argument('--headless')

options.add_argument(
    "--user-agent=Mozilla/5.0 (iPhone; CPU iPhone OS 17_0 like Mac OS X) "
    "AppleWebKit/605.1.15 (KHTML, like Gecko) Version/17.0 Mobile/15E148 Safari/604.1"
)

browser = webdriver.Chrome(options=options)

In [20]:
# Setup search parameters
city = "Montreal"
city_link = "Montréal"
product = "Gaming PC"
min_price = 0
max_price = 5000
days_listed = 10
MAX_SCROLLS = 4

In [21]:
# Setup base URL
url = f'https://www.facebook.com/marketplace/{city}/search?query={product}&minPrice={min_price}&maxPrice={max_price}&daysSinceListed={days_listed}&exact=false'

# Visit website
browser.get(url)

# Close login pop-up
try:
    close_button = browser.find_element(By.XPATH, "//div[@aria-label='Close']")
    close_button.click()
    print("Login prompt closed successfully...")
except:
    print("Login prompt not closed correctly...")
    pass

Login prompt not closed correctly...


In [ ]:
# Scroll down to load all results
wait = WebDriverWait(browser, 10) # pause execution until condition is true w/ timotu limit

last_height = int(browser.execute_script("return document.body.scrollHeight"))
MIN_DELTA = 50  # minimum height change to count as "new content"

for i in range(MAX_SCROLLS):
    browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(0.1)

    try:
        wait.until(
            lambda d: int(d.execute_script("return document.body.scrollHeight")) - last_height > MIN_DELTA
        )
        new_height = int(browser.execute_script("return document.body.scrollHeight"))
        print("Scroller successful:", new_height)
        last_height = new_height

    except TimeoutException:
        print("No further increase in scrollHeight. Stopping.")
        break

print("Finished Scrolling...")

'''
last_height = browser.execute_script("return document.body.scrollHeight")

for i in range(MAX_SCROLLS):
    # Scroll down to bottom
    browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(2)
    
    # Calculate new scroll height and compare with last scroll height
    new_height = browser.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height

print("Finished scrolling, all results loaded...")
'''


Scroller successful: 3468
Scroller successful: 6499
Scroller successful: 9529
Scroller successful: 12520
Finished Scrolling...


'\nlast_height = browser.execute_script("return document.body.scrollHeight")\n\nfor i in range(MAX_SCROLLS):\n    # Scroll down to bottom\n    browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")\n    time.sleep(2)\n\n    # Calculate new scroll height and compare with last scroll height\n    new_height = browser.execute_script("return document.body.scrollHeight")\n    if new_height == last_height:\n        break\n    last_height = new_height\n\nprint("Finished scrolling, all results loaded...")\n'

In [23]:
# Retrieve the HTML
html = browser.page_source

soup = BeautifulSoup(html, 'html.parser')

browser.close()

In [24]:
# Find link elements
links = soup.find_all('a')

# Filter on product keyword only
product_links = [
    link for link in links
    if product.lower() in link.text.lower()
    #if city_link.lower() in link.text.lower()
]

product_data = []

for product_link in product_links:
    url = product_link.get('href')
    if not url:
        continue
    text = '\n'.join(product_link.stripped_strings)
    product_data.append({'url': url, 'text': text})

#print(len(product_links), "links matched")
#print(len(product_data), "items collected")

product_data

[{'url': '/marketplace/item/1202289165053637/?ref=search&referral_code=null&referral_story_type=post&__tn__=!%3AD',
  'text': '$650\n$750\nGaming PC AMD 5700x / MSI 3070 GPU\nSan Francisco, CA'},
 {'url': '/marketplace/item/4211399239137993/?ref=search&referral_code=null&referral_story_type=post&__tn__=!%3AD',
  'text': '$649\nGaming PC\nSan Francisco, CA'},
 {'url': '/marketplace/item/1342125647645733/?ref=search&referral_code=null&referral_story_type=post&__tn__=!%3AD',
  'text': '$600\n$700\nHigh-End Gaming PC – i7-9700K – TITAN X 12 GB – 32 GB RAM – 2 TB SSD – 1440p / 4K\nSan Francisco, CA'},
 {'url': '/marketplace/item/810831765100576/?ref=search&referral_code=null&referral_story_type=post&__tn__=!%3AD',
  'text': '$800\nGaming Pc\nSan Francisco, CA'},
 {'url': '/marketplace/item/811833064921549/?ref=search&referral_code=null&referral_story_type=post&__tn__=!%3AD',
  'text': '$450\nRGB GTX 1660 Super Gaming Pc\nSan Francisco, CA'},
 {'url': '/marketplace/item/2347540209049347/?ref

In [25]:
# Create an empty list to store product data
extracted_data = []

for item in product_data:
    lines = item['text'].split('\n')

    # Regular expression to find numeric values
    numeric_pattern = re.compile('\d[\d,.]*')
    
    # Extracting prices
    # Iterate through lines to find the first line with numbers
    for line in lines:
        match = numeric_pattern.search(line)
        if match:    
            # Extract the first numeric value found
            price_str = match.group()
            # Convert price to float (handle commas)
            price = float(price_str.replace(',',''))
            break
    '''
    if price:
        print(f"Price extracted: {price}")
    else:
        print("price not found")
    '''
    # Extract title
    title = lines[-2]

    # Extract location
    location = lines[-1]

    # Add extracted data to a list of dictionaries
    extracted_data.append({
        'title': title,
        'price': price,
        'location': location,
        'url': re.sub(r'\?.*', '', item['url'])
    })

<>:8: SyntaxWarning: invalid escape sequence '\d'
<>:8: SyntaxWarning: invalid escape sequence '\d'
/var/folders/xn/w_xnr1cn1jq5m_6rgnybnsc40000gn/T/ipykernel_35566/1369819200.py:8: SyntaxWarning: invalid escape sequence '\d'
  numeric_pattern = re.compile('\d[\d,.]*')


In [26]:
extracted_data

[{'title': 'Gaming PC AMD 5700x / MSI 3070 GPU',
  'price': 650.0,
  'location': 'San Francisco, CA',
  'url': '/marketplace/item/1202289165053637/'},
 {'title': 'Gaming PC',
  'price': 649.0,
  'location': 'San Francisco, CA',
  'url': '/marketplace/item/4211399239137993/'},
 {'title': 'High-End Gaming PC – i7-9700K – TITAN X 12 GB – 32 GB RAM – 2 TB SSD – 1440p / 4K',
  'price': 600.0,
  'location': 'San Francisco, CA',
  'url': '/marketplace/item/1342125647645733/'},
 {'title': 'Gaming Pc',
  'price': 800.0,
  'location': 'San Francisco, CA',
  'url': '/marketplace/item/810831765100576/'},
 {'title': 'RGB GTX 1660 Super Gaming Pc',
  'price': 450.0,
  'location': 'San Francisco, CA',
  'url': '/marketplace/item/811833064921549/'},
 {'title': 'Gaming PC with a 3070ti',
  'price': 1000.0,
  'location': 'San Francisco, CA',
  'url': '/marketplace/item/2347540209049347/'},
 {'title': 'Dell OptiPlex Gaming PC',
  'price': 250.0,
  'location': 'San Francisco, CA',
  'url': '/marketplace/i